<a href="https://colab.research.google.com/github/ANTONY-AGNEL-SHYJU/fixature/blob/main/right_and_left_eeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
# Install the 'mne' library for reading EEG data files
!pip install mne

import mne
import numpy as np
import pywt
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE PROCESSING FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def extract_features(signal):
    """Extracts features from a signal using Discrete Wavelet Transform (DWT)."""
    coeffs = pywt.wavedec(signal, 'db4', level=5)
    features = []
    for c in coeffs:
        features.append(np.mean(c))
        features.append(np.std(c))
        features.append(np.min(c))
        features.append(np.max(c))
    return np.array(features)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD, PROCESS, AND EXTRACT FEATURES FROM DATA
# ==============================================================================

files = ['S001R04.edf', 'S001R08.edf', 'S001R12.edf']
all_features = []
all_labels = []

print("\nStarting data processing...")
for file in files:
    print(f"Processing file: {file}")

    raw = mne.io.read_raw_edf(file, preload=True, verbose=False)

    events, event_id_map = mne.events_from_annotations(raw, event_id={'T0': 1, 'T1': 2, 'T2': 3}, verbose=False)

    epochs_event_id = {'left': 2, 'right': 3}

    # ==========================================================================
    # FIX: Use the exact channel names with two periods at the end
    # ==========================================================================
    picks_channels = ['C3..', 'C4..']

    epochs = mne.Epochs(raw, events, epochs_event_id, tmin=0.0, tmax=4.0, proj=False,
                        picks=picks_channels, baseline=None, preload=True, verbose=False)

    labels = epochs.events[:, -1]
    data = epochs.get_data()

    for i in range(len(labels)):
        filtered_c3 = bandpass_filter(data[i, 0, :])
        filtered_c4 = bandpass_filter(data[i, 1, :])

        features_c3 = extract_features(filtered_c3)
        features_c4 = extract_features(filtered_c4)

        combined_features = np.concatenate([features_c3, features_c4])

        all_features.append(combined_features)
        all_labels.append(labels[i])

X = np.array(all_features)
y = np.array(all_labels)

print("\n✅ Data processing complete!")
print(f"Shape of feature matrix (X): {X.shape}")
print(f"Shape of labels vector (y): {y.shape}")

# ==============================================================================
# STEP 4: TRAIN AND TEST THE SVM MODEL
# ==============================================================================

print("\nStarting model training...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print("\n✅ Model training complete!")
print("="*30)
print(f"ACCURACY ON TEST DATA: {accuracy * 100:.2f}%")
print("="*30)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Starting data processing...
Processing file: S001R04.edf
Processing file: S001R08.edf
Processing file: S001R12.edf

✅ Data processing complete!
Shape of feature matrix (X): (45, 48)
Shape of labels vector (y): (45,)

Starting model training...

✅ Model training complete!
ACCURACY ON TEST DATA: 55.56%


In [12]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
# Install the 'mne' library for reading EEG data files
!pip install mne

import mne
import numpy as np
import pywt
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE PROCESSING FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def extract_features(signal):
    """Extracts features from a signal using Discrete Wavelet Transform (DWT)."""
    coeffs = pywt.wavedec(signal, 'db4', level=5)
    features = []
    for c in coeffs:
        features.append(np.mean(c))
        features.append(np.std(c))
        features.append(np.min(c))
        features.append(np.max(c))
    return np.array(features)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD, PROCESS, AND EXTRACT FEATURES FROM DATA
# ==============================================================================

# Using data from ONLY Subject 2
files = [
    'S002R04.edf', 'S002R08.edf', 'S002R12.edf'
]

all_features = []
all_labels = []

print("\nStarting data processing...")
for file in files:
    try:
        print(f"Processing file: {file}")
        raw = mne.io.read_raw_edf(file, preload=True, verbose=False)
        events, event_id_map = mne.events_from_annotations(raw, event_id={'T0': 1, 'T1': 2, 'T2': 3}, verbose=False)
        epochs_event_id = {'left': 2, 'right': 3}

        picks_channels = ['C3..', 'C4..']

        epochs = mne.Epochs(raw, events, epochs_event_id, tmin=0.0, tmax=4.0, proj=False,
                            picks=picks_channels, baseline=None, preload=True, verbose=False)

        labels = epochs.events[:, -1]
        data = epochs.get_data()

        for i in range(len(labels)):
            filtered_c3 = bandpass_filter(data[i, 0, :])
            filtered_c4 = bandpass_filter(data[i, 1, :])

            features_c3 = extract_features(filtered_c3)
            features_c4 = extract_features(filtered_c4)

            combined_features = np.concatenate([features_c3, features_c4])

            all_features.append(combined_features)
            all_labels.append(labels[i])
    except Exception as e:
        print(f"Could not process {file}. Error: {e}")


X = np.array(all_features)
y = np.array(all_labels)

print("\n✅ Data processing complete!")
print(f"Shape of feature matrix (X): {X.shape}")
print(f"Shape of labels vector (y): {y.shape}")

# ==============================================================================
# STEP 4: TRAIN AND TEST THE SVM MODEL
# ==============================================================================

print("\nStarting model training...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ==========================================================================
# IMPROVEMENT: Tune the model for Subject 2 by using a more flexible kernel
# ==========================================================================
model = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print("\n✅ Model training complete!")
print("="*30)
print(f"ACCURACY ON TEST DATA (Subject 2 Tuned): {accuracy * 100:.2f}%")
print("="*30)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Starting data processing...
Processing file: S002R04.edf
Processing file: S002R08.edf
Processing file: S002R12.edf

✅ Data processing complete!
Shape of feature matrix (X): (45, 48)
Shape of labels vector (y): (45,)

Starting model training...

✅ Model training complete!
ACCURACY ON TEST DATA (Subject 2 Tuned): 55.56%


In [13]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
import pywt
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE PROCESSING FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def extract_features(signal):
    coeffs = pywt.wavedec(signal, 'db4', level=5)
    features = []
    for c in coeffs:
        features.append(np.mean(c))
        features.append(np.std(c))
        features.append(np.min(c))
        features.append(np.max(c))
    return np.array(features)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD, PROCESS, AND EXTRACT FEATURES FROM DATA
# ==============================================================================

# Using data from a single, tuned subject (e.g., Subject 2)
files = ['S002R04.edf', 'S002R08.edf', 'S002R12.edf']
all_features = []
all_labels = []

print("\nStarting data processing...")
for file in files:
    try:
        print(f"Processing file: {file}")
        raw = mne.io.read_raw_edf(file, preload=True, verbose=False)
        events, event_id_map = mne.events_from_annotations(raw, event_id={'T0': 1, 'T1': 2, 'T2': 3}, verbose=False)
        epochs_event_id = {'left': 2, 'right': 3}

        # ==========================================================================
        # IMPROVEMENT: Use 8 channels over the motor cortex instead of 2
        # ==========================================================================
        picks_channels = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

        epochs = mne.Epochs(raw, events, epochs_event_id, tmin=0.0, tmax=4.0, proj=False,
                            picks=picks_channels, baseline=None, preload=True, verbose=False)

        labels = epochs.events[:, -1]
        data = epochs.get_data() # Shape is (n_trials, n_channels, n_times)

        # Process each epoch (trial)
        for i in range(len(labels)):
            trial_features = []
            # Loop through each channel for the current trial
            for chan_idx in range(data.shape[1]):
                channel_data = data[i, chan_idx, :]
                filtered_channel = bandpass_filter(channel_data)
                channel_features = extract_features(filtered_channel)
                trial_features.append(channel_features)

            # Combine all channel features into one long vector for this trial
            combined_features = np.concatenate(trial_features)

            all_features.append(combined_features)
            all_labels.append(labels[i])
    except Exception as e:
        print(f"Could not process {file}. Error: {e}")

X = np.array(all_features)
y = np.array(all_labels)

print("\n✅ Data processing complete!")
print(f"Shape of feature matrix (X): {X.shape}") # Shape will now be (45, 192)
print(f"Shape of labels vector (y): {y.shape}")

# ==============================================================================
# STEP 4: TRAIN AND TEST THE SVM MODEL
# ==============================================================================

print("\nStarting model training...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print("\n✅ Model training complete!")
print("="*30)
print(f"ACCURACY ON TEST DATA (8 Channels): {accuracy * 100:.2f}%")
print("="*30)
#i used dwt for this,it looks at each channel speperately so i dont want this, i want to look at all channels simultaneously


✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Starting data processing...
Processing file: S002R04.edf
Processing file: S002R08.edf
Processing file: S002R12.edf

✅ Data processing complete!
Shape of feature matrix (X): (45, 192)
Shape of labels vector (y): (45,)

Starting model training...

✅ Model training complete!
ACCURACY ON TEST DATA (8 Channels): 55.56%


In [14]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Import the CSP algorithm from MNE
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE PREPROCESSING FUNCTION
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD AND PREPARE EPOCHS
# ==============================================================================

# Using data from a single, tuned subject (Subject 2)
files = ['S002R04.edf', 'S002R08.edf', 'S002R12.edf']

all_epochs_data = []
all_epochs_labels = []

print("\nStarting data processing...")
for file in files:
    try:
        print(f"Processing file: {file}")
        raw = mne.io.read_raw_edf(file, preload=True, verbose=False)
        events, event_id_map = mne.events_from_annotations(raw, event_id={'T0': 1, 'T1': 2, 'T2': 3}, verbose=False)
        epochs_event_id = {'left': 2, 'right': 3}

        # We will use the same 8 channels as before
        picks_channels = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

        epochs = mne.Epochs(raw, events, epochs_event_id, tmin=0.5, tmax=4.0, proj=False,
                            picks=picks_channels, baseline=(0.5, 0.5), preload=True, verbose=False)

        # Apply the bandpass filter to the epoched data
        epochs.apply_function(bandpass_filter, verbose=False)

        all_epochs_data.append(epochs.get_data())
        all_epochs_labels.append(epochs.events[:, -1])
    except Exception as e:
        print(f"Could not process {file}. Error: {e}")

# Combine data from all files
X = np.concatenate(all_epochs_data)
y = np.concatenate(all_epochs_labels)

print("\n✅ Data processing complete!")
print(f"Shape of data matrix (X): {X.shape}") # Shape is (n_trials, n_channels, n_times)
print(f"Shape of labels vector (y): {y.shape}")

# ==============================================================================
# STEP 4: TRAIN AND TEST THE CSP + SVM PIPELINE
# ==============================================================================

print("\nStarting model training with CSP...")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create the CSP object. n_components is the number of virtual channels to create.
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# Create the SVM classifier
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)

# Create the full pipeline
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Test the pipeline's accuracy
accuracy = pipeline.score(X_test, y_test)

print("\n✅ Model training complete!")
print("="*30)
print(f"ACCURACY ON TEST DATA (CSP + SVM): {accuracy * 100:.2f}%")
print("="*30)
#This is where i used common spatial patterns instead of dwt

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Starting data processing...
Processing file: S002R04.edf
Processing file: S002R08.edf
Processing file: S002R12.edf

✅ Data processing complete!
Shape of data matrix (X): (45, 8, 561)
Shape of labels vector (y): (45,)

Starting model training with CSP...
Computing rank from data with rank=None
    Using tolerance 7.4e-06 (2.2e-16 eps * 8 dim * 4.1e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.

✅ Model training complete!
ACCURACY ON TEST DATA (CSP + SVM): 66.67%


In [15]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
import pywt
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Import the CSP algorithm from MNE
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE PREPROCESSING FUNCTION
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD AND PREPARE EPOCHS
# ==============================================================================

# Using data from a single, tuned subject (Subject 2)
files = ['S002R04.edf', 'S002R08.edf', 'S002R12.edf']

all_epochs_data = []
all_epochs_labels = []

print("\nStarting data processing...")
for file in files:
    try:
        print(f"Processing file: {file}")
        raw = mne.io.read_raw_edf(file, preload=True, verbose=False)
        events, event_id_map = mne.events_from_annotations(raw, event_id={'T0': 1, 'T1': 2, 'T2': 3}, verbose=False)
        epochs_event_id = {'left': 2, 'right': 3}

        picks_channels = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

        epochs = mne.Epochs(raw, events, epochs_event_id, tmin=0.5, tmax=4.0, proj=False,
                            picks=picks_channels, baseline=(0.5, 0.5), preload=True, verbose=False)

        epochs.apply_function(bandpass_filter, verbose=False)

        all_epochs_data.append(epochs.get_data())
        all_epochs_labels.append(epochs.events[:, -1])
    except Exception as e:
        print(f"Could not process {file}. Error: {e}")

X = np.concatenate(all_epochs_data)
y = np.concatenate(all_epochs_labels)

print("\n✅ Data processing complete!")
print(f"Shape of data matrix (X): {X.shape}")
print(f"Shape of labels vector (y): {y.shape}")

# ==============================================================================
# STEP 4: AUTOMATE MODEL TUNING WITH GRIDSEARCHCV
# ==============================================================================

print("\nStarting automated model tuning with GridSearchCV...")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create the pipeline with CSP and SVM
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
svm = SVC(probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])

# Define the grid of parameters to search
param_grid = {
    'CSP__n_components': [2, 4, 6],
    'SVM__kernel': ['linear', 'rbf'],
    'SVM__C': [0.1, 1, 10, 100]
}

# Create and run the GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print("\n✅ Automated tuning complete!")
print(f"Best parameters found: {grid_search.best_params_}")

# Test the best found model on the held-out test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)

print("\n✅ Final model evaluation complete!")
print("="*40)
print(f"ACCURACY ON TEST DATA (GridSearchCV Tuned): {accuracy * 100:.2f}%")
print("="*40)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Starting data processing...
Processing file: S002R04.edf
Processing file: S002R08.edf
Processing file: S002R12.edf

✅ Data processing complete!
Shape of data matrix (X): (45, 8, 561)
Shape of labels vector (y): (45,)

Starting automated model tuning with GridSearchCV...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Computing rank from data with rank=None
    Using tolerance 7.4e-06 (2.2e-16 eps * 8 dim * 4.1e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.

✅ Automated tuning complete!
Best parameters found: {'CSP__n_components': 6, 'SVM__C': 10, 'SVM__kernel': 'rbf'}

✅ Final model evaluation complete!
ACCURACY ON TEST DATA (GridSearchCV Tuned): 66.67%


In [17]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD AND COMBINE DATA FOR ALL FOUR COMMANDS
# ==============================================================================
print("\nStarting data processing for 4 commands...")

SUBJECT_ID = 2
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# Task 2: Imagine left/right fist (for 'left' and 'right' commands)
task2_runs = [4, 8, 12]
X_left, y_left = get_epochs_for_task(SUBJECT_ID, task2_runs, {'T1': 2}, PICKS_CHANNELS)
X_right, y_right = get_epochs_for_task(SUBJECT_ID, task2_runs, {'T2': 3}, PICKS_CHANNELS)

# Task 4: Imagine both feet (for 'forward' command)
task4_runs = [6, 10, 14]
X_forward, y_forward = get_epochs_for_task(SUBJECT_ID, task4_runs, {'T2': 3}, PICKS_CHANNELS) # T2 in task 4 is 'both feet'
y_forward[:] = 4 # Relabel this class to '4' for forward

# All Tasks: Rest state (for 'stop' command)
all_task_runs = task2_runs + task4_runs
X_stop, y_stop = get_epochs_for_task(SUBJECT_ID, all_task_runs, {'T0': 1}, PICKS_CHANNELS)
y_stop[:] = 1 # Keep label '1' for stop

# Combine all data into final X and y arrays
X = np.concatenate([X_left, X_right, X_forward, X_stop])
y = np.concatenate([y_left, y_right, y_forward, y_stop])

print("\n✅ Data processing complete!")
print(f"Total samples: {len(y)}")
print(f"Class labels found: {np.unique(y)}")

# ==============================================================================
# STEP 4: TRAIN AND TUNE THE FOUR-CLASS MODEL
# ==============================================================================
print("\nStarting automated model tuning for 4-class problem...")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create the base pipeline (CSP + SVM)
csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])

# Use OneVsRestClassifier to handle the multi-class problem
ovr_classifier = OneVsRestClassifier(pipeline)

# Define the grid of parameters to search
# Note the syntax: 'estimator__' + 'pipeline_step__' + 'parameter'
param_grid = {
    'estimator__SVM__kernel': ['rbf'],
    'estimator__SVM__C': [1, 10, 100],
    'estimator__CSP__n_components': [6, 8]
}

# Create and run the GridSearchCV
grid_search = GridSearchCV(ovr_classifier, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print("\n✅ Automated tuning complete!")
print(f"Best parameters found: {grid_search.best_params_}")

best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)

print("\n✅ Final model evaluation complete!")
print("="*50)
print(f"FINAL ACCURACY ON TEST DATA (4-CLASS MODEL): {accuracy * 100:.2f}%")
print("="*50)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Starting data processing for 4 commands...

✅ Data processing complete!
Total samples: 156
Class labels found: [1 2 3 4]

Starting automated model tuning for 4-class problem...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Computing rank from data with rank=None
    Using tolerance 1.4e-05 (2.2e-16 eps * 8 dim * 7.9e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.4e-05 (2.2e-16 eps * 8 dim * 7.9e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMP

In [18]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD PRE-TRAINING AND FINE-TUNING DATASETS
# ==============================================================================
print("\nLoading datasets for Transfer Learning...")

SUBJECT_ID = 2
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_ff, y_actual_ff = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS) # T1=both fists, T2=both feet

# Combine all actual movement data for pre-training
X_pretrain = np.concatenate([X_actual_lr, X_actual_ff])
y_pretrain = np.concatenate([y_actual_lr, y_actual_ff])
print(f"Loaded {len(y_pretrain)} samples for pre-training.")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (TARGET USER) ---
imagined_left_right_runs = [4, 8, 12]
X_finetune, y_finetune = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
print(f"Loaded {len(y_finetune)} samples for fine-tuning.")

# Split the fine-tuning data for final testing
X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting Transfer Learning process...")

# Define the model pipeline
csp = CSP(n_components=6, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])

# --- PHASE 1: PRE-TRAINING ---
print("Phase 1: Pre-training model on ACTUAL movement data...")
pipeline.fit(X_pretrain, y_pretrain)

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
pipeline.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = pipeline.score(X_test_ft, y_test_ft)

print("\n✅ Transfer Learning complete!")
print("="*50)
print(f"FINAL ACCURACY ON TEST DATA (Transfer Learning): {accuracy * 100:.2f}%")
print("="*50)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for Transfer Learning...
Loaded 90 samples for pre-training.
Loaded 45 samples for fine-tuning.

Starting Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 1.3e-05 (2.2e-16 eps * 8 dim * 7.1e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Phase 2: Fine-tuning model on IMAGINED movement data...
Computing rank from data with rank=None
    Using tolerance 6.8e-06 (2.2e-16 eps * 8 dim * 3.8e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating cl

In [20]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR TRANSFER LEARNING
# ==============================================================================
print("\nLoading datasets for 4-Command Transfer Learning...")

SUBJECT_ID = 2
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_ff, y_actual_ff = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T2': 4}, PICKS_CHANNELS) # T2 in task 3 is 'both feet' -> forward

X_pretrain = np.concatenate([X_actual_lr, X_actual_ff])
y_pretrain = np.concatenate([y_actual_lr, y_actual_ff])
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual movements).")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (FOR FINE-TUNING AND TESTING) ---
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(SUBJECT_ID, imagined_fists_feet_runs, {'T2': 4}, PICKS_CHANNELS)
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

X_finetune = np.concatenate([X_img_lr, X_img_fwd, X_img_stop])
y_finetune = np.concatenate([y_img_lr, y_img_fwd, y_img_stop])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined movements).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting 4-Command Transfer Learning process...")

# Define the base pipeline and wrap it in a OneVsRestClassifier
csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING ---
print("Phase 1: Pre-training model on ACTUAL movement data...")
ovr_classifier.fit(X_pretrain, y_pretrain)

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ 4-Command Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Command Transfer Learning): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Transfer Learning...
Could not process S002R05.edf. Error: No matching events found for T2 (event id 4)
Could not process S002R09.edf. Error: No matching events found for T2 (event id 4)
Could not process S002R13.edf. Error: No matching events found for T2 (event id 4)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 1 dimension(s)

In [21]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR TRANSFER LEARNING (ROBUST VERSION)
# ==============================================================================
print("\nLoading datasets for 4-Command Transfer Learning...")

SUBJECT_ID = 2
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_ff, y_actual_ff = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T2': 4}, PICKS_CHANNELS)

# --- FIX: Build lists of non-empty arrays to concatenate ---
pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_ff.size > 0:
    pretrain_data_list.append(X_actual_ff)
    pretrain_labels_list.append(y_actual_ff)

X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual movements).")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (FOR FINE-TUNING AND TESTING) ---
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(SUBJECT_ID, imagined_fists_feet_runs, {'T2': 4}, PICKS_CHANNELS)
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

X_finetune = np.concatenate([X_img_lr, X_img_fwd, X_img_stop])
y_finetune = np.concatenate([y_img_lr, y_img_fwd, y_img_stop])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined movements).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting 4-Command Transfer Learning process...")

csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING (only if data is available) ---
if X_pretrain.size > 0:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Training/Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ 4-Command Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Command Transfer Learning): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Transfer Learning...
Could not process S002R05.edf. Error: No matching events found for T2 (event id 4)
Could not process S002R09.edf. Error: No matching events found for T2 (event id 4)
Could not process S002R13.edf. Error: No matching events found for T2 (event id 4)
Loaded 45 samples for pre-training (actual movements).
Could not process S002R06.edf. Error: No matching events found for T2 (event id 4)
Could not process S002R10.edf. Error: No matching events found for T2 (event id 4)
Could not process S002R14.edf. Error: No matching events found for T2 (event id 4)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 1 dimension(s)

In [22]:
#with just left right nd stop  # ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR A THREE-COMMAND MODEL
# ==============================================================================
print("\nLoading datasets for 3-Command Transfer Learning...")

SUBJECT_ID = 2
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL LEFT/RIGHT MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
X_pretrain, y_pretrain = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual left/right).")

# --- PHASE 2 DATA: IMAGINED LEFT/RIGHT/STOP (FOR FINE-TUNING) ---
imagined_left_right_runs = [4, 8, 12]
X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T0': 1}, PICKS_CHANNELS)

X_finetune = np.concatenate([X_img_lr, X_img_stop])
y_finetune = np.concatenate([y_img_lr, y_img_stop])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined left, right, stop).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 3-COMMAND MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting 3-Command Transfer Learning process...")

csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING (only if data is available) ---
if X_pretrain.size > 0:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ 3-Command Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (3-Command Transfer Learning): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 3-Command Transfer Learning...
Loaded 45 samples for pre-training (actual left/right).
Loaded 90 samples for fine-tuning (imagined left, right, stop).

Starting 3-Command Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 8.6e-06 (2.2e-16 eps * 8 dim * 4.8e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Phase 2: Fine-tuning model on IMAGINED movement data...
Computing rank from data with rank=None
    Using tolerance 1e-05 (2.2e-16 eps * 8 dim * 5.6e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 

In [25]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR A THREE-COMMAND MODEL
# ==============================================================================
print("\nLoading datasets for 3-Command Transfer Learning (4-Channel)...")

SUBJECT_ID = 2
# --- MODIFICATION: Use only the 4 most critical channels ---
PICKS_CHANNELS = ['C3..', 'Cz..', 'C4..', 'Cpz.']

# --- PHASE 1 DATA: ACTUAL LEFT/RIGHT MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
X_pretrain, y_pretrain = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual left/right).")

# --- PHASE 2 DATA: IMAGINED LEFT/RIGHT/STOP (FOR FINE-TUNING) ---
imagined_left_right_runs = [4, 8, 12]
X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T0': 1}, PICKS_CHANNELS)

X_finetune = np.concatenate([X_img_lr, X_img_stop])
y_finetune = np.concatenate([y_img_lr, y_img_stop])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined left, right, stop).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 3-COMMAND MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting 3-Command Transfer Learning process (4-Channel)...")

csp = CSP(n_components=4, reg=None, log=True, norm_trace=False) # n_components cannot be larger than n_channels
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING (only if data is available) ---
if X_pretrain.size > 0:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ 3-Command Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Channel, 3-Command Model): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 3-Command Transfer Learning (4-Channel)...
Loaded 45 samples for pre-training (actual left/right).
Loaded 90 samples for fine-tuning (imagined left, right, stop).

Starting 3-Command Transfer Learning process (4-Channel)...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 3.1e-06 (2.2e-16 eps * 4 dim * 3.5e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Phase 2: Fine-tuning model on IMAGINED movement data...
Computing rank from data with rank=None
    Using tolerance 3.7e-06 (2.2e-16 eps * 4 dim * 4.2e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Red

In [26]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR TRANSFER LEARNING (SUBJECT 5)
# ==============================================================================
print("\nLoading datasets for 4-Command Transfer Learning...")

# --- MODIFICATION: Set the Subject ID to 5 ---
SUBJECT_ID = 5
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_ff, y_actual_ff = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T2': 4}, PICKS_CHANNELS) # T2 in task 3 is 'both feet' -> forward

# Build lists of non-empty arrays to concatenate
pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_ff.size > 0:
    pretrain_data_list.append(X_actual_ff)
    pretrain_labels_list.append(y_actual_ff)

X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual movements).")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (FOR FINE-TUNING AND TESTING) ---
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(SUBJECT_ID, imagined_fists_feet_runs, {'T2': 4}, PICKS_CHANNELS)
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

# Build lists of non-empty arrays for fine-tuning data
finetune_data_list = []
finetune_labels_list = []
if X_img_lr.size > 0:
    finetune_data_list.append(X_img_lr)
    finetune_labels_list.append(y_img_lr)
if X_img_fwd.size > 0:
    finetune_data_list.append(X_img_fwd)
    finetune_labels_list.append(y_img_fwd)
if X_img_stop.size > 0:
    finetune_data_list.append(X_img_stop)
    finetune_labels_list.append(y_img_stop)

X_finetune = np.concatenate(finetune_data_list) if finetune_data_list else np.array([])
y_finetune = np.concatenate(finetune_labels_list) if finetune_labels_list else np.array([])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined movements).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting 4-Command Transfer Learning process...")

csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING (only if data is available) ---
if X_pretrain.size > 0:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ 4-Command Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Command Transfer Learning, Subject 5): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Transfer Learning...
Could not process S005R05.edf. Error: No matching events found for T2 (event id 4)
Could not process S005R09.edf. Error: No matching events found for T2 (event id 4)
Could not process S005R13.edf. Error: No matching events found for T2 (event id 4)
Loaded 45 samples for pre-training (actual movements).
Could not process S005R06.edf. Error: No matching events found for T2 (event id 4)
Could not process S005R10.edf. Error: No matching events found for T2 (event id 4)
Could not process S005R14.edf. Error: No matching events found for T2 (event id 4)
Loaded 135 samples for fine-tuning (imagined movements).

Starting 4-Command Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 5.9e-06 (2.2e-16 eps * 8 dim * 3.3e+09  max singular value)
    Estimated rank (data): 8
    dat

In [27]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR TRANSFER LEARNING
# ==============================================================================
print("\nLoading datasets for 4-Command Transfer Learning...")

SUBJECT_ID = 2
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS) # T2 in task 3 is 'both feet'
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel this class to '4' for forward

pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_fwd.size > 0:
    pretrain_data_list.append(X_actual_fwd)
    pretrain_labels_list.append(y_actual_fwd)

X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual movements).")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (FOR FINE-TUNING AND TESTING) ---
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel this class to '4' for forward
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

X_finetune = np.concatenate([X_img_lr, X_img_fwd, X_img_stop])
y_finetune = np.concatenate([y_img_lr, y_img_fwd, y_img_stop])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined movements).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting 4-Command Transfer Learning process...")

csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING (only if data is available) ---
if X_pretrain.size > 0:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ 4-Command Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Command Transfer Learning): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Transfer Learning...
Loaded 69 samples for pre-training (actual movements).
Loaded 156 samples for fine-tuning (imagined movements).

Starting 4-Command Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 8 dim * 6.3e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 8 dim * 6.3e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimati

In [28]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR TRANSFER LEARNING (4-CHANNEL)
# ==============================================================================
print("\nLoading datasets for 4-Command Transfer Learning (4-Channel)...")

SUBJECT_ID = 5 # Using Subject 5 as requested
# --- MODIFICATION: Use only the 4 most critical channels ---
PICKS_CHANNELS = ['C3..', 'Cz..', 'C4..', 'Cpz.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel this class to '4' for forward

pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_fwd.size > 0:
    pretrain_data_list.append(X_actual_fwd)
    pretrain_labels_list.append(y_actual_fwd)

X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual movements).")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (FOR FINE-TUNING AND TESTING) ---
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel this class to '4' for forward
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

finetune_data_list = []
finetune_labels_list = []
if X_img_lr.size > 0:
    finetune_data_list.append(X_img_lr)
    finetune_labels_list.append(y_img_lr)
if X_img_fwd.size > 0:
    finetune_data_list.append(X_img_fwd)
    finetune_labels_list.append(y_img_fwd)
if X_img_stop.size > 0:
    finetune_data_list.append(X_img_stop)
    finetune_labels_list.append(y_img_stop)

X_finetune = np.concatenate(finetune_data_list) if finetune_data_list else np.array([])
y_finetune = np.concatenate(finetune_labels_list) if finetune_labels_list else np.array([])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined movements).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL (4-CHANNEL) USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting 4-Command Transfer Learning process (4-Channel)...")

# --- MODIFICATION: Adjust CSP components for 4 channels ---
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING ---
if X_pretrain.size > 0:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ 4-Command Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Channel, 4-Command Model, Subject 5): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Transfer Learning (4-Channel)...
Loaded 68 samples for pre-training (actual movements).
Loaded 157 samples for fine-tuning (imagined movements).

Starting 4-Command Transfer Learning process (4-Channel)...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 2.6e-06 (2.2e-16 eps * 4 dim * 2.9e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.6e-06 (2.2e-16 eps * 4 dim * 2.9e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using 

In [29]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR TRANSFER LEARNING
# ==============================================================================
print("\nLoading datasets for Transfer Learning...")

SUBJECT_ID = 2 # Change this to the subject you want to test
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel this class to '4' for forward

pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_fwd.size > 0:
    pretrain_data_list.append(X_actual_fwd)
    pretrain_labels_list.append(y_actual_fwd)

X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual movements).")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (FOR FINE-TUNING AND TESTING) ---
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14] # ADDING THESE RUNS AS REQUESTED
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel this class to '4' for forward
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

finetune_data_list = []
finetune_labels_list = []
if X_img_lr.size > 0:
    finetune_data_list.append(X_img_lr)
    finetune_labels_list.append(y_img_lr)
if X_img_fwd.size > 0:
    finetune_data_list.append(X_img_fwd)
    finetune_labels_list.append(y_img_fwd)
if X_img_stop.size > 0:
    finetune_data_list.append(X_img_stop)
    finetune_labels_list.append(y_img_stop)

X_finetune = np.concatenate(finetune_data_list) if finetune_data_list else np.array([])
y_finetune = np.concatenate(finetune_labels_list) if finetune_labels_list else np.array([])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined movements).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE MODEL USING TRANSFER LEARNING
# ==============================================================================
print("\nStarting Transfer Learning process...")

csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING (only if data is available) ---
if X_pretrain.size > 0 and len(np.unique(y_pretrain)) > 1:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing or single-class data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (Transfer Learning, Subject {SUBJECT_ID}): {accuracy * 100:.2f}%")
print("="*60)


✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for Transfer Learning...
Loaded 69 samples for pre-training (actual movements).
Loaded 156 samples for fine-tuning (imagined movements).

Starting Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 8 dim * 6.3e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 8 dim * 6.3e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covarianc

In [30]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD INTER-SUBJECT DATA FOR 4 COMMANDS
# ==============================================================================
print("\nLoading datasets for 4-Command Inter-Subject Transfer Learning...")

PRETRAIN_SUBJECT_ID = 5
TARGET_SUBJECT_ID = 2
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL MOVEMENT FROM PRE-TRAIN SUBJECT (S5) ---
print(f"Loading pre-training data from Subject {PRETRAIN_SUBJECT_ID}...")
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]
X_actual_lr, y_actual_lr = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel
X_pretrain = np.concatenate([X_actual_lr, X_actual_fwd])
y_pretrain = np.concatenate([y_actual_lr, y_actual_fwd])
print(f"Loaded {len(y_pretrain)} samples for pre-training.")

# --- PHASE 2 DATA: IMAGINED MOVEMENT FROM TARGET SUBJECT (S2) ---
print(f"Loading fine-tuning data from Subject {TARGET_SUBJECT_ID}...")
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14] # Including runs for 'forward'
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel
X_img_stop, y_img_stop = get_epochs_for_task(TARGET_SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

X_finetune = np.concatenate([X_img_lr, X_img_fwd, X_img_stop])
y_finetune = np.concatenate([y_img_lr, y_img_fwd, y_img_stop])
print(f"Loaded {len(y_finetune)} samples for fine-tuning.")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL USING INTER-SUBJECT TRANSFER LEARNING
# ==============================================================================
print("\nStarting Inter-Subject Transfer Learning process...")

csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING ---
if X_pretrain.size > 0:
    print(f"Phase 1: Pre-training model on ACTUAL movement data from Subject {PRETRAIN_SUBJECT_ID}...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing pre-train data.")

# --- PHASE 2: FINE-TUNING ---
print(f"Phase 2: Fine-tuning model on IMAGINED movement data from Subject {TARGET_SUBJECT_ID}...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ Inter-Subject Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (S{PRETRAIN_SUBJECT_ID} -> S{TARGET_SUBJECT_ID}): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Inter-Subject Transfer Learning...
Loading pre-training data from Subject 5...
Loaded 68 samples for pre-training.
Loading fine-tuning data from Subject 2...
Loaded 156 samples for fine-tuning.

Starting Inter-Subject Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data from Subject 5...
Computing rank from data with rank=None
    Using tolerance 7.2e-06 (2.2e-16 eps * 8 dim * 4.1e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7.2e-06 (2.2e-16 eps * 8 dim * 4.1e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing da

In [31]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD INTER-SUBJECT DATA FOR 4 COMMANDS (4-CHANNEL)
# ==============================================================================
print("\nLoading datasets for 4-Command Inter-Subject Transfer Learning...")

PRETRAIN_SUBJECT_ID = 5
TARGET_SUBJECT_ID = 2
# --- MODIFICATION: Use only the 4 most critical channels ---
PICKS_CHANNELS = ['C3..', 'Cz..', 'C4..', 'Cpz.']

# --- PHASE 1 DATA: ACTUAL MOVEMENT FROM PRE-TRAIN SUBJECT (S5) ---
print(f"Loading pre-training data from Subject {PRETRAIN_SUBJECT_ID}...")
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]
X_actual_lr, y_actual_lr = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel
X_pretrain = np.concatenate([X_actual_lr, X_actual_fwd])
y_pretrain = np.concatenate([y_actual_lr, y_actual_fwd])
print(f"Loaded {len(y_pretrain)} samples for pre-training.")

# --- PHASE 2 DATA: IMAGINED MOVEMENT FROM TARGET SUBJECT (S2) ---
print(f"Loading fine-tuning data from Subject {TARGET_SUBJECT_ID}...")
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel
X_img_stop, y_img_stop = get_epochs_for_task(TARGET_SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

X_finetune = np.concatenate([X_img_lr, X_img_fwd, X_img_stop])
y_finetune = np.concatenate([y_img_lr, y_img_fwd, y_img_stop])
print(f"Loaded {len(y_finetune)} samples for fine-tuning.")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL USING INTER-SUBJECT TRANSFER LEARNING
# ==============================================================================
print("\nStarting Inter-Subject Transfer Learning process...")

# --- MODIFICATION: Adjust CSP components for 4 channels ---
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING ---
if X_pretrain.size > 0:
    print(f"Phase 1: Pre-training model on ACTUAL movement data from Subject {PRETRAIN_SUBJECT_ID}...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing pre-train data.")

# --- PHASE 2: FINE-TUNING ---
print(f"Phase 2: Fine-tuning model on IMAGINED movement data from Subject {TARGET_SUBJECT_ID}...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ Inter-Subject Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Channel, S{PRETRAIN_SUBJECT_ID} -> S{TARGET_SUBJECT_ID}): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Inter-Subject Transfer Learning...
Loading pre-training data from Subject 5...
Loaded 68 samples for pre-training.
Loading fine-tuning data from Subject 2...
Loaded 156 samples for fine-tuning.

Starting Inter-Subject Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data from Subject 5...
Computing rank from data with rank=None
    Using tolerance 2.6e-06 (2.2e-16 eps * 4 dim * 2.9e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.6e-06 (2.2e-16 eps * 4 dim * 2.9e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing da

In [32]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD INTER-SUBJECT DATA
# ==============================================================================
print("\nLoading datasets for Inter-Subject Transfer Learning...")

# --- MODIFICATION: Set Pre-train to S2 and Target to S5 ---
PRETRAIN_SUBJECT_ID = 2
TARGET_SUBJECT_ID = 5
PICKS_CHANNELS = ['C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']

# --- PHASE 1 DATA: ACTUAL MOVEMENT FROM PRE-TRAIN SUBJECT (S2) ---
print(f"Loading pre-training data from Subject {PRETRAIN_SUBJECT_ID}...")
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]
X_actual_lr, y_actual_lr = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel
pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_fwd.size > 0:
    pretrain_data_list.append(X_actual_fwd)
    pretrain_labels_list.append(y_actual_fwd)
X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training.")

# --- PHASE 2 DATA: IMAGINED MOVEMENT FROM TARGET SUBJECT (S5) ---
print(f"Loading fine-tuning data from Subject {TARGET_SUBJECT_ID}...")
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel
X_img_stop, y_img_stop = get_epochs_for_task(TARGET_SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

finetune_data_list = []
finetune_labels_list = []
if X_img_lr.size > 0:
    finetune_data_list.append(X_img_lr)
    finetune_labels_list.append(y_img_lr)
if X_img_fwd.size > 0:
    finetune_data_list.append(X_img_fwd)
    finetune_labels_list.append(y_img_fwd)
if X_img_stop.size > 0:
    finetune_data_list.append(X_img_stop)
    finetune_labels_list.append(y_img_stop)
X_finetune = np.concatenate(finetune_data_list) if finetune_data_list else np.array([])
y_finetune = np.concatenate(finetune_labels_list) if finetune_labels_list else np.array([])
print(f"Loaded {len(y_finetune)} samples for fine-tuning.")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE MODEL USING INTER-SUBJECT TRANSFER LEARNING
# ==============================================================================
print("\nStarting Inter-Subject Transfer Learning process...")

csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING ---
if X_pretrain.size > 0:
    print(f"Phase 1: Pre-training model on ACTUAL movement data from Subject {PRETRAIN_SUBJECT_ID}...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing pre-train data.")

# --- PHASE 2: FINE-TUNING ---
print(f"Phase 2: Fine-tuning model on IMAGINED movement data from Subject {TARGET_SUBJECT_ID}...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ Inter-Subject Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (S{PRETRAIN_SUBJECT_ID} -> S{TARGET_SUBJECT_ID}): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for Inter-Subject Transfer Learning...
Loading pre-training data from Subject 2...
Loaded 69 samples for pre-training.
Loading fine-tuning data from Subject 5...
Loaded 157 samples for fine-tuning.

Starting Inter-Subject Transfer Learning process...
Phase 1: Pre-training model on ACTUAL movement data from Subject 2...
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 8 dim * 6.3e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 8 dim * 6.3e+09  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank fr

In [33]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD INTER-SUBJECT DATA (4-CHANNEL)
# ==============================================================================
print("\nLoading datasets for 4-Command Inter-Subject Transfer Learning (4-Channel)...")

# --- MODIFICATION: Switched Pre-train to S2 and Target to S5 ---
PRETRAIN_SUBJECT_ID = 2
TARGET_SUBJECT_ID = 5
PICKS_CHANNELS = ['C3..', 'Cz..', 'C4..', 'Cpz.']

# --- PHASE 1 DATA: ACTUAL MOVEMENT FROM PRE-TRAIN SUBJECT (S2) ---
print(f"Loading pre-training data from Subject {PRETRAIN_SUBJECT_ID}...")
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]
X_actual_lr, y_actual_lr = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(PRETRAIN_SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel
pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_fwd.size > 0:
    pretrain_data_list.append(X_actual_fwd)
    pretrain_labels_list.append(y_actual_fwd)
X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training.")

# --- PHASE 2 DATA: IMAGINED MOVEMENT FROM TARGET SUBJECT (S5) ---
print(f"Loading fine-tuning data from Subject {TARGET_SUBJECT_ID}...")
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs
X_img_lr, y_img_lr = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(TARGET_SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel
X_img_stop, y_img_stop = get_epochs_for_task(TARGET_SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)
finetune_data_list = []
finetune_labels_list = []
if X_img_lr.size > 0:
    finetune_data_list.append(X_img_lr)
    finetune_labels_list.append(y_img_lr)
if X_img_fwd.size > 0:
    finetune_data_list.append(X_img_fwd)
    finetune_labels_list.append(y_img_fwd)
if X_img_stop.size > 0:
    finetune_data_list.append(X_img_stop)
    finetune_labels_list.append(y_img_stop)
X_finetune = np.concatenate(finetune_data_list) if finetune_data_list else np.array([])
y_finetune = np.concatenate(finetune_labels_list) if finetune_labels_list else np.array([])
print(f"Loaded {len(y_finetune)} samples for fine-tuning.")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE 4-COMMAND MODEL (4-CHANNEL) USING INTER-SUBJECT TRANSFER LEARNING
# ==============================================================================
print("\nStarting Inter-Subject Transfer Learning process (4-Channel)...")

csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING ---
if X_pretrain.size > 0:
    print(f"Phase 1: Pre-training model on ACTUAL movement data from Subject {PRETRAIN_SUBJECT_ID}...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing pre-train data.")

# --- PHASE 2: FINE-TUNING ---
print(f"Phase 2: Fine-tuning model on IMAGINED movement data from Subject {TARGET_SUBJECT_ID}...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ Inter-Subject Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (S{PRETRAIN_SUBJECT_ID} -> S{TARGET_SUBJECT_ID}): {accuracy * 100:.2f}%")
print("="*60)

✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for 4-Command Inter-Subject Transfer Learning (4-Channel)...
Loading pre-training data from Subject 2...
Loaded 69 samples for pre-training.
Loading fine-tuning data from Subject 5...
Loaded 157 samples for fine-tuning.

Starting Inter-Subject Transfer Learning process (4-Channel)...
Phase 1: Pre-training model on ACTUAL movement data from Subject 2...
Computing rank from data with rank=None
    Using tolerance 4.1e-06 (2.2e-16 eps * 4 dim * 4.6e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 4.1e-06 (2.2e-16 eps * 4 dim * 4.6e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 

In [1]:
# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# ==============================================================================
!pip install mne

import mne
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from mne.decoding import CSP

print("✅ Libraries installed and imported successfully.")

# ==============================================================================
# STEP 2: DEFINE HELPER FUNCTIONS
# ==============================================================================

def bandpass_filter(data, lowcut=8.0, highcut=30.0, fs=160.0, order=5):
    """Applies a bandpass filter to the data."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def get_epochs_for_task(subject_id, run_numbers, event_id, picks_channels):
    """Helper function to load and extract epochs for specific runs and events."""
    all_epochs_data = []
    all_epochs_labels = []

    for run in run_numbers:
        filename = f'S{subject_id:03d}R{run:02d}.edf'
        try:
            raw = mne.io.read_raw_edf(filename, preload=True, verbose=False)
            events, _ = mne.events_from_annotations(raw, verbose=False)

            epochs = mne.Epochs(raw, events, event_id, tmin=0.5, tmax=4.0, proj=False,
                                picks=picks_channels, baseline=None, preload=True, verbose=False)
            epochs.apply_function(bandpass_filter, verbose=False)

            all_epochs_data.append(epochs.get_data())
            all_epochs_labels.append(epochs.events[:, -1])
        except Exception as e:
            print(f"Could not process {filename}. Error: {e}")

    if not all_epochs_data:
        return np.array([]), np.array([])

    return np.concatenate(all_epochs_data), np.concatenate(all_epochs_labels)

print("✅ Helper functions defined.")

# ==============================================================================
# STEP 3: LOAD DATA FOR TRANSFER LEARNING (4-CHANNEL)
# ==============================================================================
print("\nLoading datasets for Intra-Subject Transfer Learning (4-Channel)...")

SUBJECT_ID = 5 # Change this to the subject you want to test
# --- MODIFICATION: Use only the 4 most critical channels ---
PICKS_CHANNELS = ['C3..', 'Cz..', 'C4..', 'Cpz.']

# --- PHASE 1 DATA: ACTUAL PHYSICAL MOVEMENT (FOR PRE-TRAINING) ---
actual_left_right_runs = [3, 7, 11]
actual_fists_feet_runs = [5, 9, 13]

X_actual_lr, y_actual_lr = get_epochs_for_task(SUBJECT_ID, actual_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_actual_fwd, y_actual_fwd = get_epochs_for_task(SUBJECT_ID, actual_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_actual_fwd.size > 0: y_actual_fwd[:] = 4 # Relabel this class to '4' for forward

pretrain_data_list = []
pretrain_labels_list = []
if X_actual_lr.size > 0:
    pretrain_data_list.append(X_actual_lr)
    pretrain_labels_list.append(y_actual_lr)
if X_actual_fwd.size > 0:
    pretrain_data_list.append(X_actual_fwd)
    pretrain_labels_list.append(y_actual_fwd)

X_pretrain = np.concatenate(pretrain_data_list) if pretrain_data_list else np.array([])
y_pretrain = np.concatenate(pretrain_labels_list) if pretrain_labels_list else np.array([])
print(f"Loaded {len(y_pretrain)} samples for pre-training (actual movements).")

# --- PHASE 2 DATA: IMAGINED MOVEMENT (FOR FINE-TUNING AND TESTING) ---
imagined_left_right_runs = [4, 8, 12]
imagined_fists_feet_runs = [6, 10, 14]
all_imagined_runs = imagined_left_right_runs + imagined_fists_feet_runs

X_img_lr, y_img_lr = get_epochs_for_task(SUBJECT_ID, imagined_left_right_runs, {'T1': 2, 'T2': 3}, PICKS_CHANNELS)
X_img_fwd, y_img_fwd = get_epochs_for_task(SUBJECT_ID, imagined_fists_feet_runs, {'T2': 3}, PICKS_CHANNELS)
if y_img_fwd.size > 0: y_img_fwd[:] = 4 # Relabel
X_img_stop, y_img_stop = get_epochs_for_task(SUBJECT_ID, all_imagined_runs, {'T0': 1}, PICKS_CHANNELS)

finetune_data_list = []
finetune_labels_list = []
if X_img_lr.size > 0:
    finetune_data_list.append(X_img_lr)
    finetune_labels_list.append(y_img_lr)
if X_img_fwd.size > 0:
    finetune_data_list.append(X_img_fwd)
    finetune_labels_list.append(y_img_fwd)
if X_img_stop.size > 0:
    finetune_data_list.append(X_img_stop)
    finetune_labels_list.append(y_img_stop)

X_finetune = np.concatenate(finetune_data_list) if finetune_data_list else np.array([])
y_finetune = np.concatenate(finetune_labels_list) if finetune_labels_list else np.array([])
print(f"Loaded {len(y_finetune)} samples for fine-tuning (imagined movements).")

X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_finetune, y_finetune, test_size=0.3, random_state=42, stratify=y_finetune)

# ==============================================================================
# STEP 4: TRAIN THE MODEL USING INTRA-SUBJECT TRANSFER LEARNING
# ==============================================================================
print("\nStarting Intra-Subject Transfer Learning process (4-Channel)...")

# --- MODIFICATION: Adjust CSP components for 4 channels ---
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
pipeline = Pipeline([('CSP', csp), ('SVM', svm)])
ovr_classifier = OneVsRestClassifier(pipeline)

# --- PHASE 1: PRE-TRAINING ---
if X_pretrain.size > 0 and len(np.unique(y_pretrain)) > 1:
    print("Phase 1: Pre-training model on ACTUAL movement data...")
    ovr_classifier.fit(X_pretrain, y_pretrain)
else:
    print("Phase 1: Skipping pre-training due to missing or single-class data.")

# --- PHASE 2: FINE-TUNING ---
print("Phase 2: Fine-tuning model on IMAGINED movement data...")
ovr_classifier.fit(X_train_ft, y_train_ft)

# --- FINAL EVALUATION ---
accuracy = ovr_classifier.score(X_test_ft, y_test_ft)

print("\n✅ Intra-Subject Transfer Learning complete!")
print("="*60)
print(f"FINAL ACCURACY ON TEST DATA (4-Channel, Subject {SUBJECT_ID}): {accuracy * 100:.2f}%")
print("="*60)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.8 MB/s eta 0:00:00
✅ Libraries installed and imported successfully.
✅ Helper functions defined.

Loading datasets for Intra-Subject Transfer Learning (4-Channel)...
Loaded 68 samples for pre-training (actual movements).
Loaded 157 samples for fine-tuning (imagined movements).

Starting Intra-Subject Transfer Learning process (4-Channel)...
Phase 1: Pre-training model on ACTUAL movement data...
Computing rank from data with rank=None
    Using tolerance 2.6e-06 (2.2e-16 eps * 4 dim * 2.9e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.6e-06 (2.2e-16 eps * 4 dim * 2.9e+09  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels 

In [8]:
import mne

# Load one of your files to inspect it
file_to_inspect = 'S001R04.edf'
raw = mne.io.read_raw_edf(file_to_inspect, verbose=False)

# Print the list of all channel names
print(raw.ch_names)

['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']
